In [1]:
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard
from keras import optimizers

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

import pickle
import time

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/Simple CNN keras/dataset2/train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/Simple CNN keras/dataset2/train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/Simple CNN keras/dataset2/test",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)

Found 7307 images belonging to 2 classes.
Found 1826 images belonging to 2 classes.
Found 899 images belonging to 2 classes.


In [3]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))

model = Sequential()

model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(BatchNormalization())

model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          294976    
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)             

In [4]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/Simple CNN keras/logs/{}".format(NAME))

#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

filepath="/home/trojan/Desktop/dimentia/heatmap/weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = checkpoint

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[callbacks_list, tensorboard])

model.save('MRI_vgg16.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10
913/913 [==============================] - 156s 170ms/step - loss: 0.6977 - accuracy: 0.5025 - val_loss: 0.6717 - val_accuracy: 0.5033

Epoch 00001: val_accuracy improved from -inf to 0.50329, saving model to /home/trojan/Desktop/dimentia/heatmap/weights-improvement-01-0.50.hdf5

Epoch 2/10
913/913 [==============================] - 155s 170ms/step - loss: 0.6941 - accuracy: 0.4969 - val_loss: 0.6885 - val_accuracy: 0.4945

Epoch 00002: val_accuracy did not improve from 0.50329
Epoch 3/10
913/913 [==============================] - 156s 170ms/step - loss: 0.6930 - accuracy: 0.5069 - val_loss: 0.6956 - val_accuracy: 0.5347

Epoch 00003: val_accuracy improved from 0.50329 to 0.53465, saving model to /home/trojan/Desktop/dimentia/heatmap/weights-improvement-03-0.51.hdf5
Epoch 4/10
913/913 [==============================] - 158s 173ms/step - loss: 0.6876 - accuracy: 0.5380 - val_loss:

In [ ]:
model.evaluate_generator(generator=test_generator,
steps=STEP_SIZE_TEST)